In [4]:
import json, pickle
import glob, os

import numpy as np
import pandas as pd

from qiskit import transpile
from qiskit import execute
from qiskit.providers.fake_provider import FakeLima
from qiskit.primitives import Estimator
from qiskit.circuit.random import random_circuit

import torch
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.nn.functional import dropout

from torch_geometric.nn import GCNConv, global_mean_pool, Linear, ChebConv, SAGEConv
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

from blackwater.data.loaders.exp_val import CircuitGraphExpValMitigationDataset
from blackwater.data.generators.exp_val import exp_value_generator
from blackwater.data.utils import generate_random_pauli_sum_op
from blackwater.library.ngem.estimator import ngem

from qiskit.quantum_info import random_clifford

import random
from qiskit.circuit.library import HGate, SdgGate
from qiskit.circuit import ClassicalRegister

from blackwater.data.utils import (
    generate_random_pauli_sum_op,
    create_estimator_meas_data,
    circuit_to_graph_data_json,
    get_backend_properties_v1,
    encode_pauli_sum_op,
    create_meas_data_from_estimators
)
from blackwater.data.generators.exp_val import ExpValueEntry
from blackwater.metrics.improvement_factor import improvement_factor, Trial, Problem

from qiskit_aer import AerSimulator, QasmSimulator
from qiskit.providers.fake_provider import FakeMontreal, FakeLima, FakeBelem, FakeGuadalupe

from zne import zne, ZNEStrategy
from zne.noise_amplification import *
from zne.extrapolation import *

from torch_geometric.nn import (
    GCNConv,
    TransformerConv,
    GATv2Conv,
    global_mean_pool,
    Linear,
    ChebConv,
    SAGEConv,
    ASAPooling,
    dense_diff_pool,
    avg_pool_neighbor_x
)
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.utils import to_dense_adj, to_dense_batch

from qiskit import QuantumCircuit
from qiskit.circuit.library import U3Gate, CZGate, PhaseGate, CXGate
from mbd_utils import construct_random_clifford, cal_z_exp, calc_imbalance, cal_all_z_exp, construct_mbl_circuit, generate_disorder, random_clifford_circuit, modify_and_add_noise_to_model
from gnn import ExpValCircuitGraphModel
from qiskit.quantum_info import Clifford
from qiskit_aer.noise import depolarizing_error, coherent_unitary_error, NoiseModel

from functools import partial

/home/sreyas/.conda/envs/mlqem/lib/python3.9/site-packages/torch_geometric/typing.py:110: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: /home/sreyas/.conda/envs/mlqem/lib/python3.9/site-packages/zmq/backend/cython/../../../../.././libstdc++.so.6: version `GLIBCXX_3.4.32' not found (required by /home/sreyas/.conda/envs/mlqem/lib/python3.9/site-packages/torch_sparse/_version_cpu.so)
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "


In [5]:
# backend = FakeGuadalupe()
backend = FakeMontreal()
properties = get_backend_properties_v1(backend)

## Local
backend_ideal = QasmSimulator() # Noiseless
backend_noisy = AerSimulator.from_backend(backend) # Noisy

run_config_ideal = {'shots': 10000, 'backend': backend_ideal, 'name': 'ideal'}
run_config_noisy = {'shots': 10000, 'backend': backend_noisy, 'name': 'noisy'}

num_qubit = 4

In [6]:
print(backend.configuration().num_qubits)

27


In [7]:
def fix_random_seed(seed=0):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    print(f'random seed fixed to {seed}')

In [20]:
def get_zne_expval(circ,
                   obs: str,
                   extrapolator,
                   backend,
                   noise_factors = (1, 3),
                   amplifier=LocalFoldingAmplifier(gates_to_fold=2),
                   shots:int=10000,
                   ) -> float:
    """Returns the zero-noise expectation values for a list of observables `obs` of a circuit `circ`
    Inputs:
    obs (list[str]): Observables in Endian convention (e.g. 'IIZ' is the Z operator on the 0th qubit) for the PHYSICAL qubits
    mapping (list[int]): Mapping between logical and physical qubits, e.g. mapping[0] is the physical qubit for logical qubit 0
    """
    observable = SparsePauliOp(obs)

    ZNEEstimator = zne(BackendEstimator)
    estimator = ZNEEstimator(backend=backend)

    zne_strategy = ZNEStrategy(
        noise_factors=noise_factors,
        noise_amplifier=amplifier,
        extrapolator=extrapolator
    )

    job = estimator.run(circ, observable, shots=shots, zne_strategy=zne_strategy)
    values = job.result().values
    # we use a different convention where +1 is the eigenvalue for |1> state, because cal_z_exp assumes 0 is -Z and 1 is +Z
    values *= -1

    return values.tolist()

def get_measurement_qubits(qc, num_qubit):
    measurement_qubits = []
    for measurement in range(num_qubit - 1, -1, -1):
        instruction = qc[-1 - measurement]
        if instruction[0].name == 'measure':
            measurement_qubits.append(instruction[1][0].index)
    return measurement_qubits


def get_all_circuit_meas_mapping(circs):
    out = []
    for circ in circs:
        mapping = get_measurement_qubits(circ, 16)
        mapping = mapping[::-1]
        for i in range(16):
            if i not in mapping:
                mapping = mapping + [i]
        out.append(mapping)
    return out

In [2]:
!pip install mitiq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.3/169.3 kB 2.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 5.1 MB/s eta 0:00:0000:0100:01m
Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl (29 kB)


In [4]:
import qiskit
from qiskit_aer import QasmSimulator
from qiskit_ibm_runtime import QiskitRuntimeService

from mitiq import zne
from mitiq.interface.mitiq_qiskit.qiskit_utils import initialized_depolarizing_noise
USE_REAL_HARDWARE = False


linear_factory = zne.inference.LinearFactory(scale_factors=[1.0, 1.5, 2.0, 2.5, 3.0])
mitigated = zne.execute_with_zne(circuit, ibmq_executor, factory=linear_factory)
print(f"Mitigated result {mitigated:.3f}")

mitigated = zne.execute_with_zne(circuit, ibmq_executor, scale_noise=zne.scaling.fold_global)
print(f"Mitigated result {mitigated:.3f}")

scale_factors = [1., 1.5, 2., 2.5, 3.]
folded_circuits = [
        zne.scaling.fold_gates_at_random(circuit, scale)
        for scale in scale_factors
]

# Check that the circuit depth is (approximately) scaled as expected
for j, c in enumerate(folded_circuits):
    print(f"Number of gates of folded circuit {j} scaled by: {len(c) / len(circuit):.3f}")

shots = 8192

# Transpile the circuit so it can be properly run
exec_circuit = qiskit.transpile(
    folded_circuits,
    backend=backend,
    basis_gates=noise_model.basis_gates if noise_model else None,
    optimization_level=0, # Important to preserve folded gates.
)

# Run the circuit
job = backend.run(exec_circuit, shots=shots)


# Get the results
all_counts = [job.result().get_counts(i) for i in range(len(folded_circuits))]
expectation_values = [counts.get("0") / shots for counts in all_counts]
print(f"Expectation values:\n{expectation_values}")

# Plot the expectation values
print("Unmitigated expectation value:", round(expectation_values[0], 3))
zero_noise_value = zne.ExpFactory.extrapolate(scale_factors, expectation_values, asymptote=0.5)
print(f"Extrapolated zero-noise value:", round(zero_noise_value, 3))

TypeError: execute_with_zne() got an unexpected keyword argument 'folding'

In [21]:
from qiskit.transpiler.exceptions import TranspilerError
from qiskit.circuit.exceptions import CircuitError
import scipy

def generate_data(circuits, save_path: str, run_config_n, do_zne=False):
    entries = []
    n = circuits[0].num_qubits
    for circuit in tqdm(circuits):
        success = False
        while not success:
            try:
                success = True
            except (scipy.linalg.LinAlgError, TranspilerError, np.linalg.LinAlgError, CircuitError) as e:
                print(f"Ran into an error:, {e}")

        print('!')
        job_ideal = execute(circuit, **run_config_ideal)
        print('!!')
        job_noisy = execute(circuit, **run_config_n)


        counts_ideal = job_ideal.result().get_counts()
        counts_noisy = job_noisy.result().get_counts()

        ideal_exp_val = cal_z_exp(counts_ideal)
        noisy_exp_val = cal_z_exp(counts_noisy)
        mitigated = None

        print('!!!')
        if do_zne:
            degree = 1
            extrapolator = PolynomialExtrapolator(degree=degree)
            mitigated = []

            # ob_list has single Z measurement on the first qubit in the last position, i.e., 'ZIIII' is measuring the top qubit (Non-endian notation)
            ob_list = np.array(['I'*i+'Z'+'I'*(16-i-1) for i in range(16)][::-1])
            mapping = get_all_circuit_meas_mapping(circuit)
            ob_list = ob_list[np.array(mapping)].tolist()
            for obs in ob_list:
                ############################ Change the backend #######################################
                mitigated += get_zne_expval(circuit, obs, extrapolator, backend_noisy)
                #######################################################################################
            print([round(x, 3) for x in mitigated])

        to_append = dict()
        to_append['circuit'] = circuit
        to_append['noisy_exp_val'] = noisy_exp_val
        to_append['ideal_exp_val'] = ideal_exp_val
        to_append['zne_mitigated'] = mitigated
        entries.append(to_append)

    path = f"{save_path}/q{n}.pk"
    with open(path, "wb") as f:
        pickle.dump(entries, f)

In [22]:
import re
def load_circuits(data_dir, f_ext='.json'):
    circuits = []
    ideal_exp_vals = []
    noisy_exp_vals = []
    data_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith(f_ext)]
    steps = []
    for data_file in tqdm(data_files, leave=True):
        if f_ext == '.json':
            for entry in json.load(open(data_file, 'r')):
                circuits.append(QuantumCircuit.from_qasm_str(entry['circuit']))
                ideal_exp_vals.append(entry['ideal_exp_value'])
                noisy_exp_vals.append(entry['noisy_exp_values'])
        elif f_ext == '.pk':
            step = int(re.search(r'\d+', data_file.split('/')[-1]).group())
            for entry in pickle.load(open(data_file, 'rb')):
                steps.append(step)
                circuits.append(entry['circuit'])
                ideal_exp_vals.append(entry['ideal_exp_value'])
                noisy_exp_vals.append(entry['noisy_exp_values'])
    return circuits, ideal_exp_vals, noisy_exp_vals, steps

circuits, ideal_exp_vals, noisy_exp_vals, steps = load_circuits('./data/s_test/val/', '.pk')
print(len(circuits))

  0%|          | 0/1 [00:00<?, ?it/s]

10


In [23]:
generate_data(circuits, './data/s_test/zne', run_config_n=run_config_noisy, do_zne=True)

  0%|          | 0/10 [00:00<?, ?it/s]

!
!!
!!!


IndexError: tuple index out of range